In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

In [44]:
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

In [36]:
import pickle
with open('./final_gu_vars.pkl', 'rb') as f:
    최종딕셔너리 = pickle.load(f)

In [ ]:
pd.set_option('display.max_columns', None)

In [37]:
rbl = pd.read_csv('region_bubble_labels.csv')

In [50]:
rbl.describe()

,Unnamed: 0,전국,서울,강북14개구,강남11개구,6개광역시,부산,대전,대구,인천,...,서울_label,강북14개구_label,강남11개구_label,6개광역시_label,부산_label,대전_label,대구_label,인천_label,광주_label,울산_label
count,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,...,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000
mean,124.000000,32.147547,36.455977,31.335018,39.171144,30.201577,29.989362,28.730905,23.284333,29.278574,...,0.281124,0.261044,0.244980,0.690763,0.216867,0.341365,0.192771,0.614458,0.289157,0.301205
std,72.024301,12.650253,13.754915,14.847687,13.250346,14.732731,17.324992,15.783562,11.483049,16.518026,...,0.809025,0.832873,0.808364,0.543246,0.838319,0.827900,0.834602,0.605712,0.796111,0.804247
min,0.000000,0.696414,0.513148,0.500587,0.556543,0.860140,1.214906,0.003592,0.420804,0.146559,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,62.000000,26.288086,31.014625,26.333798,33.051904,16.464365,8.285647,13.272448,18.446344,13.096371,...,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,0.000000
50%,124.000000,33.893627,36.245076,31.638595,39.635896,33.776026,37.396329,29.828375,24.727743,34.566810,...,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000
75%,186.000000,39.832496,45.369843,40.232197,48.254619,40.752012,42.885294,42.995608,28.382161,42.821675,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,248.000000,54.416078,60.437713,57.506151,61.938397,54.173256,54.861453,52.427260,48.458382,53.886496,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [38]:
gbl = pd.read_csv('bubble_labels.csv')   

In [53]:
gbl.describe()

,Unnamed: 0,종로구,중구,용산구,성동구,광진구,동대문구,중랑구,성북구,강북구,도봉구,노원구,은평구,서대문구,마포구,양천구,강서구,구로구,금천구,영등포구,동작구,관악구,서초구,강남구,송파구,강동구,종로구_label,중구_label,용산구_label,성동구_label,광진구_label,동대문구_label,중랑구_label,성북구_label,강북구_label,도봉구_label,노원구_label,은평구_label,서대문구_label,마포구_label,양천구_label,강서구_label,구로구_label,금천구_label,영등포구_label,동작구_label,관악구_label,서초구_label,강남구_label,송파구_label,강동구_label
count,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000
mean,124.000000,24.304053,22.150176,40.826860,36.467310,31.295495,26.820413,27.734103,24.410289,23.802208,23.748182,34.218358,27.737091,27.600864,36.514816,38.554239,31.451398,28.687349,32.530872,39.818399,35.139667,28.648782,42.933169,42.901041,42.344405,41.520818,0.433735,0.309237,0.349398,0.285141,0.277108,0.309237,0.164659,0.244980,0.160643,0.228916,0.180723,0.301205,0.277108,0.309237,0.184739,0.184739,0.228916,0.349398,0.305221,0.285141,0.220884,0.289157,0.253012,0.220884,0.216867
std,72.024301,11.388414,11.316518,13.473687,13.694061,12.899113,12.788206,13.737030,14.769194,13.698968,14.482375,17.728326,12.285480,13.095998,14.485483,13.853523,14.361099,14.948808,13.125963,14.964748,14.692347,13.806604,13.500285,12.607696,12.378054,11.392328,0.770307,0.801180,0.809725,0.839439,0.822760,0.816140,0.828546,0.823193,0.826902,0.822937,0.834893,0.843403,0.837333,0.806198,0.831589,0.846011,0.822937,0.804730,0.820118,0.824902,0.830010,0.801160,0.795806,0.825138,0.813914
min,0.000000,0.728551,-0.682007,0.441671,0.530753,0.203509,0.937948,0.126803,-0.108148,0.601201,-0.003020,0.381972,0.171983,0.562047,2.004585,0.237828,0.798487,0.418668,0.921226,0.265751,0.183965,-0.027032,0.101066,1.554471,0.074754,0.398232,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,62.000000,21.660747,18.322758,36.076123,31.444646,25.173118,21.684568,25.167005,18.046025,18.196716,16.211729,30.057353,22.031440,21.549730,30.973004,31.775750,24.320228,22.804478,27.906893,33.511659,28.683394,23.362922,38.614982,36.481038,35.661671,35.982415,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,-1.000000,0.000000
50%,124.000000,24.566692,21.819990,42.226418,36.946524,30.601052,26.213300,29.134782,23.429140,23.934028,23.019471,36.075224,26.825847,26.629678,35.414512,38.911057,32.822096,27.374234,32.533682,39.148056,35.389533,28.352828,44.095296,42.552672,41.866784,41.381335,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,186.000000,30.827837,27.727493,49.461348,46.469466,39.511984,35.053495,34.312664,34.503561,30.460609,31.103169,43.265843,32.515441,36.498886,47.145060,46.987211,39.873987,37.257305,38.813492,50.685595,45.719625,37.681682,52.637609,52.941985,50.667280,49.694815,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,248.000000,44.462604,43.411705,61.980680,58.824178,54.709050,51.046277,52.126475,52.570235,49.493610,52.579854,63.9

In [39]:
gu = ['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구',
       '성북구', '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구',
       '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구']

In [40]:
seoul_gu = gu.copy()
seoul_gu.append('서울')

In [41]:
gu_label = []
for idx in gu:
    gu_label.append(idx+'_label')

gu_df = []
for idx in seoul_gu:
    gu_df.append(idx+'_df')

In [42]:
gu_data = gbl[gu_label]
seoul_data = rbl[['날짜','서울_label']]
total_data = pd.concat([seoul_data, gu_data], axis=1)

In [45]:
param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

param_grid_xgb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 6],
    'subsample': [0.8, 1.0]
}

param_grid_svm = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto'],
}

model_params = {
    'RF': (RandomForestClassifier(), param_grid_rf),
    'XGB': (XGBClassifier(use_label_encoder=False, eval_metric='logloss'), param_grid_xgb),
    'SVM': (SVC(kernel='rbf'), param_grid_svm)
}


In [48]:
seed = 42
np.random.seed(seed)
test_scores = {'RF': {}, 'XGB': {}, 'SVM': {}}
def fit_and_evaluate(df1, df2, idx):
    df2['날짜'] = pd.to_datetime(df2['날짜'])
    label = df2[['날짜', idx+'_label']]
    data = pd.merge(df1, label, on='날짜')
    predict_set= data.iloc[244:247]
    data = data[1:244]
    data.dropna(inplace=True)
    
    X = data.drop([idx+'_label', '날짜'], axis=1)
    y = data[idx+'_label']
    predict_X = predict_set.drop([idx+'_label', '날짜'], axis=1)
    split_point = int(len(data) * 0.8)
    X_train, X_test = X[:split_point], X[split_point:]
    y_train, y_test = y[:split_point], y[split_point:]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    predict_X_scaled = scaler.transform(predict_X)
    mapping = {-1: 0, 0: 1, 1: 2}
    y_train_mapped = y_train.map(mapping)
    y_test_mapped = y_test.map(mapping)
    
    avg_valid_scores = {'RF': [], 'XGB': [], 'SVM': []}

    for name, (model, params) in model_params.items():
        grid_search = GridSearchCV(model, params, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), scoring='f1_weighted')
        grid_search.fit(X_train_scaled, y_train_mapped)
        
        best_model = grid_search.best_estimator_
        
        y_pred = best_model.predict(X_test_scaled)
        f1 = f1_score(y_test_mapped, y_pred, average='weighted')
        test_scores[name][idx] = f1
        avg_valid_scores[name].append(grid_search.best_score_)
        predictions = best_model.predict(predict_X_scaled)
        print(f"Best parameters for {name} on {idx}: {grid_search.best_params_}")
        print(f"Average Validation F1 Score for {name} on {idx}: {grid_search.best_score_}")
        print(f"Predictions for {name} on {idx}: {predictions}")

for idx in seoul_gu:  
    df_idx = idx+'_df'
    df1 = 최종딕셔너리[df_idx]
    df2 = total_data.copy()  
    fit_and_evaluate(df1, df2, idx)

print("Test F1 Scores:", test_scores)

Best parameters for RF on 종로구: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Average Validation F1 Score for RF on 종로구: 0.8623392620451444
Predictions for RF on 종로구: [2 2 2]
Best parameters for XGB on 종로구: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.8}
Average Validation F1 Score for XGB on 종로구: 0.8656380893286238
Predictions for XGB on 종로구: [2 2 2]
Best parameters for SVM on 종로구: {'C': 10, 'gamma': 'scale'}
Average Validation F1 Score for SVM on 종로구: 0.8592549228598301
Predictions for SVM on 종로구: [2 2 2]
Best parameters for RF on 중구: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Average Validation F1 Score for RF on 중구: 0.9201073672281102
Predictions for RF on 중구: [2 2 2]
Best parameters for XGB on 중구: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200, 'subsample': 1.0}
Average Validation F1 Score for XGB on 중구: 0.8927071901220508
Predictions for XGB on 중구: [2 2 2]